# iRep (bPTR) and coverage breadth analysis

In [1]:
rm(list = ls())
setwd("/mnt/c/Users/Cedric/Desktop/git_repos/blood_microbiome")
require("Rsamtools")
require(Biostrings)
require(tidyverse)
require(data.table)
require(foreach)
require(doParallel)
registerDoParallel(cores = 10)

Loading required package: Rsamtools

Loading required package: GenomeInfoDb

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors

L

In [2]:
species_df <- fread("results/decontamination/read_matrix.raw.zeroed.csv")

In [3]:
files <- list.files("results/irep_analysis/raw_output/irep_out", full.names = T)
files <- files[grepl(".tsv", files)]

morsels <- foreach(file = files) %do% {
    temp_df <- read.csv(file, sep = "\t", check.names = F)
    colnames(temp_df)[1] <- "ref_path"
    temp_df %>% 
        mutate(across(c(4:ncol(temp_df)), function(x) (as.numeric(ifelse(x == "n/a", NA, x)))),
               across(c(1:3), as.character)) %>%
        pivot_longer(!all_of(c("ref_path", "ORI", "TER")), names_to = "prefix", values_to = "bPTR") %>%
        separate(ref_path, sep = "/", into = c(rep(NA, 11), "file_name")) %>%
        separate(file_name, sep = "_", into = c("genus", "species", "suffix1", "suffix2"), remove = F) %>%
        separate(prefix, into = c(rep(NA, 6), "id"), sep = "/") %>%
        separate(id, into = c("npm_research_id"), sep = "_") %>%
        mutate(prefix = gsub(".fasta", "", file_name))
}

irep_df <- bind_rows(morsels) 


Warning message:
“Expected 1 pieces. Additional pieces discarded in 5 rows [1, 2, 3, 4, 5].”
Warning message:
“Expected 1 pieces. Additional pieces discarded in 5 rows [1, 2, 3, 4, 5].”
Warning message:
“Expected 1 pieces. Additional pieces discarded in 5 rows [1, 2, 3, 4, 5].”
Warning message:
“Expected 1 pieces. Additional pieces discarded in 5 rows [1, 2, 3, 4, 5].”
Warning message:
“Expected 1 pieces. Additional pieces discarded in 5 rows [1, 2, 3, 4, 5].”
Warning message:
“Expected 1 pieces. Additional pieces discarded in 5 rows [1, 2, 3, 4, 5].”
Warning message:
“Expected 1 pieces. Additional pieces discarded in 5 rows [1, 2, 3, 4, 5].”
Warning message:
“Expected 1 pieces. Additional pieces discarded in 5 rows [1, 2, 3, 4, 5].”
Warning message:
“Expected 1 pieces. Additional pieces discarded in 5 rows [1, 2, 3, 4, 5].”
Warning message:
“Expected 1 pieces. Additional pieces discarded in 4 rows [1, 2, 3, 4].”
Warning message:
“Expected 1 pieces. Additional pieces discarded in 5 row

## Parse coverage breadth

In [4]:
irep_df

file_name,genus,species,suffix1,suffix2,ORI,TER,npm_research_id,bPTR,prefix
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
Achromobacter_xylosoxidans_NZ_CP043820.1.fasta,Achromobacter,xylosoxidans,NZ,CP043820.1.fasta,0,0,1116-0023,NA,Achromobacter_xylosoxidans_NZ_CP043820.1
Achromobacter_xylosoxidans_NZ_CP043820.1.fasta,Achromobacter,xylosoxidans,NZ,CP043820.1.fasta,0,0,WHB3658,NA,Achromobacter_xylosoxidans_NZ_CP043820.1
Achromobacter_xylosoxidans_NZ_CP043820.1.fasta,Achromobacter,xylosoxidans,NZ,CP043820.1.fasta,0,0,WHB3674,NA,Achromobacter_xylosoxidans_NZ_CP043820.1
Achromobacter_xylosoxidans_NZ_CP043820.1.fasta,Achromobacter,xylosoxidans,NZ,CP043820.1.fasta,0,0,WHB3686,NA,Achromobacter_xylosoxidans_NZ_CP043820.1
Achromobacter_xylosoxidans_NZ_CP043820.1.fasta,Achromobacter,xylosoxidans,NZ,CP043820.1.fasta,0,0,WHB3734,NA,Achromobacter_xylosoxidans_NZ_CP043820.1
Acinetobacter_baumannii_NZ_CP043953.1.fasta,Acinetobacter,baumannii,NZ,CP043953.1.fasta,"3,869,401","1,967,227",0116-0053,1.896385,Acinetobacter_baumannii_NZ_CP043953.1
Acinetobacter_baumannii_NZ_CP043953.1.fasta,Acinetobacter,baumannii,NZ,CP043953.1.fasta,"3,869,401","1,967,227",WHB4240,NA,Acinetobacter_baumannii_NZ_CP043953.1
Acinetobacter_baumannii_NZ_CP043953.1.fasta,Acinetobacter,baumannii,NZ,CP043953.1.fasta,"3,869,401","1,967,227",WHB4251,NA,Acinetobacter_baumannii_NZ_CP043953.1
Acinetobacter_baumannii_NZ_CP043953.1.fasta,Acinetobacter,baumannii,NZ,CP043953.1.fasta,"3,869,401","1,967,227",WHB4438,NA,Acinetobacter_baumannii_NZ_CP043953.1


In [5]:
bam_path <- "results/irep_analysis/raw_output/bam_files"
ref_path <- "data/irep_data/genome_references"
prefixes <- list.files(ref_path, ".fasta", full.names = F)
prefixes <- gsub(".fasta", "", prefixes)
prefixes

[1] "Achromobacter_xylosoxidans_NZ_CP043820.1"       
 [2] "Acinetobacter_baumannii_NZ_CP043953.1"          
 [3] "Alcaligenes_faecalis_NZ_CP013119.1"             
 [4] "Aspergillus_oryzae_NC_036435.1"                 
 [5] "Corynebacterium_segmentosum_NZ_LR134408.1"      
 [6] "Cupriavidus_metallidurans_NZ_FYAX01000013.1"    
 [7] "Cutibacterium_acnes_NC_021085.1"                
 [8] "Fannyhessea_vaginae_NZ_UFSV01000001.1"          
 [9] "Fusobacterium_nucleatum_NZ_LN831027.1"          
[10] "Fusobacterium_pseudoperiodonticum_NZ_CP024702.1"
[11] "Gardnerella_vaginalis_NZ_PKJK01000001.1"        
[12] "Haemophilus_parainfluenzae_NZ_GL872339.1"       
[13] "Hepatitis_B_virus_genB_JX661488.1"              
[14] "Human_betaherpesvirus_6A_NC_001664.4"           
[15] "Human_betaherpesvirus_6B_NC_000898.1"           
[16] "Human_mastadenovirus_C_NC_001405.1"             
[17] "Lactobacillus_crispatus_NZ_CP039266.1"          
[18] "Malassezia_restricta_NC_040193.1"               
[19] "Moraxella_osloensis_NZ_CP014234.1"              
[20] "Neisseria_flavescens_NZ_CP039886.1"             
[21] "Neisseria_mucosa_NZ_UGRT01000001.1"             
[22] "Neisseria_subflava_NZ_CP039887.1"               
[23] "Prevotella_melaninogenica_NC_014370.1"          
[24] "Prevotella_oral_taxon_299_NC_022124.1"          
[25] "Pseudomonas_mendocina_NZ_CP013124.1"            
[26] "Rickettsia_Tillamook_23_NZ_CP060138.1"          
[27] "Staphylococcus_cohnii_NZ_UHDA01000001.1"        
[28] "Staphylococcus_epidermidis_NZ_CP035288.1"       
[29] "Staphylococcus_haemolyticus_NZ_CP013911.1"      
[30] "Torque_teno_virus_6_NC_014094.1"

In [6]:
morsels <- foreach (i = seq(length(prefixes))) %dopar% {
    prefix <- prefixes[i]
    bams <- list.files(str_glue("{bam_path}/{prefix}"))
    bams <- bams[!grepl(".bai", bams)]

    crumbs <- foreach (j = seq(length(bams))) %do% {
        bam <- bams[j]
        id <- str_split(bam, "_")[[1]][1]
        ref <- readDNAStringSet(str_glue("{ref_path}/{prefixes[i]}.fasta"))
        ref_length <- width(ref)
        params <- PileupParam(distinguish_strands = F, distinguish_nucleotides = F)
        pile_df <- pileup(str_glue("{bam_path}/{prefix}/{bam}"), pileupParam = params)
        
        n_covered1 <- pile_df %>%
            filter(count >= 1) %>%
            nrow()
        
        n_covered5 <- pile_df %>%
            filter(count >= 5) %>%
            nrow()

        perc_covered1 <- n_covered1 / ref_length * 100
        perc_covered5 <- n_covered5 / ref_length * 100
        
        taxon <- str_split(prefix, "_")[[1]]
            read_count <- species_df %>% 
                filter(npm_research_id == id) %>%
                select(contains(taxon[1])) %>%
                select(contains(taxon[2])) %>%
                select(!contains("monkey"))

        if (ncol(read_count) > 1) {
            read_count <- read_count %>% 
                select(contains(taxon[3]))
            if (ncol(read_count) > 1) {
            read_count <- read_count %>% 
                select(contains(paste0(" ", taxon[4])))
            }
        }
        
        if (all(dim(read_count) == 1)) {
            read_count <- deframe(read_count)
        } else {
            read_count <- NA
        }
        
        return(tibble(prefix = prefix, 
                      npm_research_id = id,
                      pairs_assigned = read_count,
                      perc_covered1 = perc_covered1,
                      perc_covered5 = perc_covered5))
    }
    
    return(bind_rows(crumbs))
}

cov_df <- bind_rows(morsels)

In [7]:
# prefixes
# prefix <- "Hepatitis_B_virus_genB_JX661488.1"    
# id <- "WHB4594"
# taxon <- str_split(prefix, "_")[[1]]
#     read_count <- species_df %>% 
#         filter(npm_research_id == id) %>%
#         select(contains(taxon[1])) %>%
#         select(contains(taxon[2])) %>%
#         select(!contains("monkey"))
    
# #     if (ncol(read_count) > 1) {
# #         read_count <- read_count %>% 
# #             select(contains(taxon[3]))
# #         if (ncol(read_count) > 1) {
# #         read_count <- read_count %>% 
# #             select(contains(paste0(" ", taxon[4])))
# #         }
# #     }

# #     if (all(dim(read_count) == 1)) {
# #         read_count <- deframe(read_count)
# #     } else {
# #         read_count <- NA
# #     }

# read_count

### Merged results

In [8]:
mapping_df <- fread("results/irep_analysis/raw_output/mapping_read_counts.csv")
mapping_df %>% filter(grepl("Prevo", prefix))

prefix,npm_research_id,pairs_mapped
<chr>,<chr>,<int>
Prevotella_melaninogenica_NC_014370.1,WHB3907,902
Prevotella_melaninogenica_NC_014370.1,WHB3939,148
Prevotella_melaninogenica_NC_014370.1,WHB5008,68
Prevotella_melaninogenica_NC_014370.1,WHB8397,130
Prevotella_melaninogenica_NC_014370.1,WHB9179,3942
Prevotella_oral_taxon_299_NC_022124.1,WHB4373,50
Prevotella_oral_taxon_299_NC_022124.1,WHB9179,1274


In [10]:
merged_df <- cov_df %>%
    left_join(mapping_df) %>%
    left_join(irep_df) %>%
    relocate(pairs_mapped, .after = 3) %>%
    mutate(pairs_mapped = pairs_mapped / 2)

merged_df

fwrite(merged_df, "results/irep_analysis/coverage_irep_results.raw.csv")

Joining, by = c("prefix", "npm_research_id")
Joining, by = c("prefix", "npm_research_id")


prefix,npm_research_id,pairs_assigned,pairs_mapped,perc_covered1,perc_covered5,file_name,genus,species,suffix1,suffix2,ORI,TER,bPTR
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
Achromobacter_xylosoxidans_NZ_CP043820.1,1116-0023,430178,429196,79.292320359,7.450282e+01,Achromobacter_xylosoxidans_NZ_CP043820.1.fasta,Achromobacter,xylosoxidans,NZ,CP043820.1.fasta,0,0,NA
Achromobacter_xylosoxidans_NZ_CP043820.1,WHB3658,385855,438419,78.389787758,7.366085e+01,Achromobacter_xylosoxidans_NZ_CP043820.1.fasta,Achromobacter,xylosoxidans,NZ,CP043820.1.fasta,0,0,NA
Achromobacter_xylosoxidans_NZ_CP043820.1,WHB3674,459561,494858,78.805094251,7.456206e+01,Achromobacter_xylosoxidans_NZ_CP043820.1.fasta,Achromobacter,xylosoxidans,NZ,CP043820.1.fasta,0,0,NA
Achromobacter_xylosoxidans_NZ_CP043820.1,WHB3686,541795,599335,79.082105806,7.525644e+01,Achromobacter_xylosoxidans_NZ_CP043820.1.fasta,Achromobacter,xylosoxidans,NZ,CP043820.1.fasta,0,0,NA
Achromobacter_xylosoxidans_NZ_CP043820.1,WHB3734,688629,769978,79.583059893,7.609348e+01,Achromobacter_xylosoxidans_NZ_CP043820.1.fasta,Achromobacter,xylosoxidans,NZ,CP043820.1.fasta,0,0,NA
Acinetobacter_baumannii_NZ_CP043953.1,0116-0053,7673,28842,79.329072139,1.697212e+01,Acinetobacter_baumannii_NZ_CP043953.1.fasta,Acinetobacter,baumannii,NZ,CP043953.1.fasta,"3,869,401","1,967,227",1.896385
Acinetobacter_baumannii_NZ_CP043953.1,WHB4240,168,210,1.316042864,1.963529e-03,Acinetobacter_baumannii_NZ_CP043953.1.fasta,Acinetobacter,baumannii,NZ,CP043953.1.fasta,"3,869,401","1,967,227",NA
Acinetobacter_baumannii_NZ_CP043953.1,WHB4251,183,233,1.460462955,4.782956e-04,Acinetobacter_baumannii_NZ_CP043953.1.fasta,Acinetobacter,baumannii,NZ,CP043953.1.fasta,"3,869,401","1,967,227",NA
Acinetobacter_baumannii_NZ_CP043953.1,WHB4438,379,453,2.824813672,3.373242e-03,Acinetobacter_baumannii_NZ_CP043953.1.fasta,Acinetobacter,baumannii,NZ,CP043953.1.fasta,"3,869,401","1,967,227",NA


In [11]:
merged_df %>% filter(grepl("oral", prefix))

prefix,npm_research_id,pairs_assigned,pairs_mapped,perc_covered1,perc_covered5,file_name,genus,species,suffix1,suffix2,ORI,TER,bPTR
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
Prevotella_oral_taxon_299_NC_022124.1,WHB4373,79,25,1.001761,0.0000000,Prevotella_oral_taxon_299_NC_022124.1.fasta,Prevotella,oral,taxon,299,0,0,NA
Prevotella_oral_taxon_299_NC_022124.1,WHB9179,2518,637,20.716208,0.3540185,Prevotella_oral_taxon_299_NC_022124.1.fasta,Prevotella,oral,taxon,299,0,0,NA


In [12]:
ori_df <- merged_df %>%
    select(prefix, ORI, TER) %>%
    filter(ORI != "0" & TER != "0") %>%
    distinct()

In [13]:
summarised_df <- merged_df %>%
    group_by(prefix) %>%
    summarise(mean_perc_covered1 = mean(perc_covered1),
              mean_perc_covered5 = mean(perc_covered5),
              min_perc_covered1 = min(perc_covered1),
              max_perc_covered1 = max(perc_covered1),
              max_bPTR = max(bPTR, na.rm = T)) %>%
    mutate(max_bPTR = ifelse(max_bPTR == -Inf, NA, max_bPTR)) %>%
    left_join(ori_df) %>%
    separate(prefix, into = c("genus", "species", "suffix1", "suffix2"), sep = "_", remove = F) %>%
    mutate(taxa = ifelse(grepl("Torque", prefix), 
                         paste(genus, species, suffix1, suffix2), 
                         paste(genus, species, suffix1)), .before = 1) %>%
    mutate(taxa = gsub(" NZ| NC", "", taxa)) %>%
    mutate(taxa = gsub("Microbacterium", "Microbacterium sp.", taxa)) %>%
    mutate(taxa = gsub("Prevotella oral taxon", "Prevotella sp. oral taxon 299", taxa)) %>%
    mutate(taxa = gsub("Rickettsia", "Rickettsia sp.", taxa)) %>%
    select(-prefix, -genus, -species, -suffix1, -suffix2) %>%
    arrange(desc(mean_perc_covered1))


fwrite(summarised_df, "results/irep_analysis/coverage_irep_results.parsed.csv")

summarised_df

Warning message in max(bPTR, na.rm = T):
“no non-missing arguments to max; returning -Inf”
Warning message in max(bPTR, na.rm = T):
“no non-missing arguments to max; returning -Inf”
Warning message in max(bPTR, na.rm = T):
“no non-missing arguments to max; returning -Inf”
Warning message in max(bPTR, na.rm = T):
“no non-missing arguments to max; returning -Inf”
Warning message in max(bPTR, na.rm = T):
“no non-missing arguments to max; returning -Inf”
Warning message in max(bPTR, na.rm = T):
“no non-missing arguments to max; returning -Inf”
Warning message in max(bPTR, na.rm = T):
“no non-missing arguments to max; returning -Inf”
Warning message in max(bPTR, na.rm = T):
“no non-missing arguments to max; returning -Inf”
Warning message in max(bPTR, na.rm = T):
“no non-missing arguments to max; returning -Inf”
Warning message in max(bPTR, na.rm = T):
“no non-missing arguments to max; returning -Inf”
Warning message in max(bPTR, na.rm = T):
“no non-missing arguments to max; returning -Inf”

taxa,mean_perc_covered1,mean_perc_covered5,min_perc_covered1,max_perc_covered1,max_bPTR,ORI,TER
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
Human betaherpesvirus 6B,98.48291943,9.544136e+01,98.34129070,98.6928951,NA,NA,NA
Human betaherpesvirus 6A,96.76555108,9.096362e+01,96.63755349,96.9826450,1.282505,"44,954","132,612"
Achromobacter xylosoxidans,79.03047361,7.481513e+01,78.38978776,79.5830599,NA,NA,NA
Hepatitis B virus,76.39191291,4.486262e+01,61.33748056,98.7558320,NA,NA,NA
Pseudomonas mendocina,71.41336595,6.739544e+01,71.12799127,71.7810391,NA,NA,NA
Cutibacterium acnes,36.15403798,3.473737e+00,15.33223211,77.4795425,NA,NA,NA
Rickettsia sp. Tillamook 23,31.85980557,2.757348e+00,0.35879756,63.3608136,1.346660,"240,832","888,370"
Alcaligenes faecalis,28.02434056,5.987024e-01,23.36010861,31.7478145,NA,NA,NA
Torque teno virus 6,18.83940621,1.705803e+01,16.08636977,21.5924426,NA,NA,NA
